In [1]:
import os 
import pandas as pd

folder_path = os.getcwd()
print(folder_path)
print(os.listdir(folder_path))

C:\Users\aorjue01\Restatement\measures
['.ipynb_checkpoints', 'Pending Measures Standard Hierarchy After Product Restatement.xlsx', 'Pending Measures Standard Hierarchy Before Product Restatement.xlsx', 'Pending Measures TT Hierarchy After Product Restatement.xlsx', 'Pending Measures TT Hierarchy Before Product Restatement.xlsx', 'restatement_check.ipynb']


In [2]:
def missing_rows(r_before, r_after):
    products_removed =  r_before[~r_before["Product"].isin(r_after["Product"])]
    products_included = r_after[~r_after["Product"].isin(r_before["Product"])]

    return products_removed["Product"], products_included["Product"]



def save_excel(name_file, sheet_name,dataframe,include_idx):
    name = name_file + ".xlsx"
    sn = sheet_name
    with pd.ExcelWriter(name, engine="openpyxl") as writer:
        dataframe.to_excel(writer, sheet_name=sn, index=include_idx)

In [3]:
def data_check(file_name_b,file_name_a,sn,ts,c_name):
    fp_after = os.path.join(folder_path , file_name_a)
    fp_before =  os.path.join(folder_path , file_name_b)
    
    raw_before = pd.read_excel(fp_before,sheet_name = sn ,skiprows=ts)
    raw_after = pd.read_excel(fp_after,sheet_name = sn ,skiprows=ts)
    rd_with_na = raw_before.shape[0] - raw_after.shape[0]

    print(f"Total row in before table: {raw_before.shape[0]}")
    print(f"Total row in after table: {raw_after.shape[0]}")
    print(f"Row difference in tables: {rd_with_na}")
    print("==="*10)
    print(raw_before.columns)
    print(raw_after.columns)
    print("==="*10)

    raw_before["Measures"] = raw_before["Measures"].str.strip()
    raw_after["Measures"] = raw_after["Measures"].str.strip()
    
    mask_before = ~raw_before[c_name].isna()
    mask_after = ~raw_after[c_name].isna()
    
    before_no_na = raw_before[mask_before]
    after_no_na = raw_after[mask_after]
    before_no_na = before_no_na.set_index("Measures") 
    after_no_na = after_no_na.set_index("Measures")
    rd_without_na = before_no_na.shape[0] - after_no_na.shape[0]
    rd_without_na
    
    print(f"Total row in before table droping N/A: {before_no_na.shape[0]}")
    print(f"Total row in after table droping N/A: {after_no_na.shape[0]}")
    print(f"Row difference after droping N/A: {rd_without_na}")
    print("==="*10)
    compare_df = before_no_na.join(after_no_na, on= "Measures",lsuffix ="_before")

    c_name_b = c_name + "_before"
    
    compare_df["difference"] = compare_df[c_name_b] - compare_df[c_name]
    compare_df["variance"] = (compare_df["difference"] / compare_df[c_name_b]) * 100
    
    comparation = compare_df[[c_name_b,c_name,"difference","variance"]]
    comparation_outrange_3p = comparation[ (comparation["variance"] > 3) | (comparation["variance"] < -3)]
    comparation_outrange_5p = comparation[ (comparation["variance"] > 5) | (comparation["variance"] < -5)]

    return raw_before , raw_after, comparation , comparation_outrange_3p,comparation_outrange_5p

In [5]:
file_name_a = "Pending Measures Standard Hierarchy After Product Restatement.xlsx"
file_name_b = "Pending Measures Standard Hierarchy Before Product Restatement.xlsx"
sheet_name = "1"
table_start = 7
column_to_check = 'Standard Hierarchy_PENDING'

raw_before, raw_after , comp , comp_or_3p, comp_or_5p = data_check(file_name_a, file_name_b, sheet_name, table_start ,column_to_check)

Total row in before table: 62
Total row in after table: 62
Row difference in tables: 0
Index(['Measures', 'Standard Hierarchy_PENDING', 'CARBONATED BEVERAGES',
       'COFFEE', 'COFFEE CREAMER - SS', 'COFFEE FILTERS', 'TEA - BAGS/LOOSE',
       'TEA - INSTANT TEA MIXES', 'TEA/COFFEE - READY-TO-DRINK',
       'COCKTAIL MIXES', 'DRINK MIXES', 'LIQUID DRINK ENHANCERS',
       'MILK FLAVORING/COCOA MIXES', 'ASEPTIC JUICES', 'BOTTLED JUICES - SS',
       'CANNED JUICES - SS', 'JUICE/DRINK CONCENTRATE - SS', 'POWDERED MILK',
       'RTD SS MILK & MILK ALTERNATIVES', 'ENERGY DRINKS', 'SPORTS DRINKS',
       'BOTTLED WATER', 'JUICES - FZ', 'BABY FOOD', 'PASTRY/DOUGHNUTS',
       'SPICES/EXTRACTS/SALT', 'NON-CHOCOLATE CANDY', 'JELLIES/JAMS',
       'SNACK BARS/GRANOLA BARS/CLUSTERS', 'CANNED/BOTTLED FRUIT',
       'PREMIXED COCKTAILS/COOLERS/HARD SELTZERS', 'CREAMS/CREAMERS', 'YOGURT',
       'TEA/COFFEE - RFG', 'N/A', 'SOAP', 'OTHER HEALTH REMEDIES', 'VITAMINS',
       'WEIGHT CONTROL', 'CARRO

In [6]:
comp

,Standard Hierarchy_PENDING_before,Standard Hierarchy_PENDING,difference,variance
Measures,,,,
Dollar Sales,6.207790e+10,6.214413e+10,-6.623255e+07,-0.106693
Unit Sales,1.453056e+10,1.453274e+10,-2.181888e+06,-0.015016
Base Dollar Sales,5.514218e+10,5.518782e+10,-4.564099e+07,-0.082770
Base Unit Sales,1.237260e+10,1.237334e+10,-7.337230e+05,-0.005930
Incremental Dollars,6.935719e+09,6.956311e+09,-2.059156e+07,-0.296892
Incremental Units,2.157958e+09,2.159406e+09,-1.448165e+06,-0.067108
Opportunity Dollars,1.987384e+07,2.149398e+07,-1.620142e+06,-8.152134
ACV Weighted Distribution,1.000000e+02,1.000000e+02,0.000000e+00,0.000000
Average Weekly ACV Distribution,9.995547e+01,9.611102e+01,3.844448e+00,3.846161


In [7]:
column_to_check = 'ASEPTIC JUICES'

raw_before, raw_after , comp , comp_or_3p, comp_or_5p = data_check(file_name_a, file_name_b, sheet_name, table_start ,column_to_check)

Total row in before table: 62
Total row in after table: 62
Row difference in tables: 0
Index(['Measures', 'Standard Hierarchy_PENDING', 'CARBONATED BEVERAGES',
       'COFFEE', 'COFFEE CREAMER - SS', 'COFFEE FILTERS', 'TEA - BAGS/LOOSE',
       'TEA - INSTANT TEA MIXES', 'TEA/COFFEE - READY-TO-DRINK',
       'COCKTAIL MIXES', 'DRINK MIXES', 'LIQUID DRINK ENHANCERS',
       'MILK FLAVORING/COCOA MIXES', 'ASEPTIC JUICES', 'BOTTLED JUICES - SS',
       'CANNED JUICES - SS', 'JUICE/DRINK CONCENTRATE - SS', 'POWDERED MILK',
       'RTD SS MILK & MILK ALTERNATIVES', 'ENERGY DRINKS', 'SPORTS DRINKS',
       'BOTTLED WATER', 'JUICES - FZ', 'BABY FOOD', 'PASTRY/DOUGHNUTS',
       'SPICES/EXTRACTS/SALT', 'NON-CHOCOLATE CANDY', 'JELLIES/JAMS',
       'SNACK BARS/GRANOLA BARS/CLUSTERS', 'CANNED/BOTTLED FRUIT',
       'PREMIXED COCKTAILS/COOLERS/HARD SELTZERS', 'CREAMS/CREAMERS', 'YOGURT',
       'TEA/COFFEE - RFG', 'N/A', 'SOAP', 'OTHER HEALTH REMEDIES', 'VITAMINS',
       'WEIGHT CONTROL', 'CARRO

In [10]:
comp.head(50)

,ASEPTIC JUICES_before,ASEPTIC JUICES,difference,variance
Measures,,,,
Dollar Sales,1.061396e+09,1.064309e+09,-2.912657e+06,-2.744176e-01
Unit Sales,2.592743e+08,2.593631e+08,-8.884005e+04,-3.426489e-02
Volume Sales,1.722690e+10,1.724146e+10,-1.456573e+07,-8.455227e-02
Base Dollar Sales,9.594100e+08,9.624583e+08,-3.048282e+06,-3.177247e-01
Base Unit Sales,2.225455e+08,2.227372e+08,-1.917473e+05,-8.616096e-02
Base Volume Sales,1.509299e+10,1.510180e+10,-8.809527e+06,-5.836835e-02
Incremental Dollars,1.019861e+08,1.018505e+08,1.356250e+05,1.329838e-01
Incremental Units,3.672881e+07,3.662590e+07,1.029073e+05,2.801814e-01
Incremental Volume,2.133910e+09,2.139666e+09,-5.756205e+06,-2.697492e-01


In [11]:
comp_or_5p

,ASEPTIC JUICES_before,ASEPTIC JUICES,difference,variance
Measures,,,,


In [12]:
comp_or_3p

,ASEPTIC JUICES_before,ASEPTIC JUICES,difference,variance
Measures,,,,
Average Weekly ACV Distribution,99.535615,95.710823,3.824791,3.842636
Total Points Cumulative Weeks,110223.623481,106063.763713,4159.859768,3.774018
Weighted Weeks,25.879260,24.884814,0.994446,3.842636
Average Weekly Total Points of Distribution,4239.370134,4079.375527,159.994606,3.774018
Avg Weekly Dollars per Store Selling,461.370993,445.855366,15.515626,3.362939
Avg Weekly Units per Store Selling,112.690772,108.866193,3.824579,3.393871
Avg Weekly Volume per Store Selling,7486.594242,7239.960997,246.633245,3.294332


In [13]:
column_to_check = 'CANNED JUICES - SS'

raw_before, raw_after , comp , comp_or_3p, comp_or_5p = data_check(file_name_a, file_name_b, sheet_name, table_start ,column_to_check)

Total row in before table: 62
Total row in after table: 62
Row difference in tables: 0
Index(['Measures', 'Standard Hierarchy_PENDING', 'CARBONATED BEVERAGES',
       'COFFEE', 'COFFEE CREAMER - SS', 'COFFEE FILTERS', 'TEA - BAGS/LOOSE',
       'TEA - INSTANT TEA MIXES', 'TEA/COFFEE - READY-TO-DRINK',
       'COCKTAIL MIXES', 'DRINK MIXES', 'LIQUID DRINK ENHANCERS',
       'MILK FLAVORING/COCOA MIXES', 'ASEPTIC JUICES', 'BOTTLED JUICES - SS',
       'CANNED JUICES - SS', 'JUICE/DRINK CONCENTRATE - SS', 'POWDERED MILK',
       'RTD SS MILK & MILK ALTERNATIVES', 'ENERGY DRINKS', 'SPORTS DRINKS',
       'BOTTLED WATER', 'JUICES - FZ', 'BABY FOOD', 'PASTRY/DOUGHNUTS',
       'SPICES/EXTRACTS/SALT', 'NON-CHOCOLATE CANDY', 'JELLIES/JAMS',
       'SNACK BARS/GRANOLA BARS/CLUSTERS', 'CANNED/BOTTLED FRUIT',
       'PREMIXED COCKTAILS/COOLERS/HARD SELTZERS', 'CREAMS/CREAMERS', 'YOGURT',
       'TEA/COFFEE - RFG', 'N/A', 'SOAP', 'OTHER HEALTH REMEDIES', 'VITAMINS',
       'WEIGHT CONTROL', 'CARRO

In [15]:
comp_or_3p

,CANNED JUICES - SS_before,CANNED JUICES - SS,difference,variance
Measures,,,,
Average Weekly ACV Distribution,99.443854,95.621947,3.821907,3.843282
Total Points Cumulative Weeks,79872.691771,76803.741967,3068.949805,3.842302
Weighted Weeks,25.855402,24.861706,0.993696,3.843282
Average Weekly Total Points of Distribution,3072.026607,2953.990076,118.036531,3.842302
Avg Weekly Dollars per Store Selling,268.235866,259.125803,9.110063,3.396288
Avg Weekly Units per Store Selling,97.517952,94.137504,3.380448,3.466488
Avg Weekly Volume per Store Selling,3306.315220,3198.024221,108.290999,3.275278


In [16]:
column_to_check = 'BOTTLED JUICES - SS'

raw_before, raw_after , comp , comp_or_3p, comp_or_5p = data_check(file_name_a, file_name_b, sheet_name, table_start ,column_to_check)

Total row in before table: 62
Total row in after table: 62
Row difference in tables: 0
Index(['Measures', 'Standard Hierarchy_PENDING', 'CARBONATED BEVERAGES',
       'COFFEE', 'COFFEE CREAMER - SS', 'COFFEE FILTERS', 'TEA - BAGS/LOOSE',
       'TEA - INSTANT TEA MIXES', 'TEA/COFFEE - READY-TO-DRINK',
       'COCKTAIL MIXES', 'DRINK MIXES', 'LIQUID DRINK ENHANCERS',
       'MILK FLAVORING/COCOA MIXES', 'ASEPTIC JUICES', 'BOTTLED JUICES - SS',
       'CANNED JUICES - SS', 'JUICE/DRINK CONCENTRATE - SS', 'POWDERED MILK',
       'RTD SS MILK & MILK ALTERNATIVES', 'ENERGY DRINKS', 'SPORTS DRINKS',
       'BOTTLED WATER', 'JUICES - FZ', 'BABY FOOD', 'PASTRY/DOUGHNUTS',
       'SPICES/EXTRACTS/SALT', 'NON-CHOCOLATE CANDY', 'JELLIES/JAMS',
       'SNACK BARS/GRANOLA BARS/CLUSTERS', 'CANNED/BOTTLED FRUIT',
       'PREMIXED COCKTAILS/COOLERS/HARD SELTZERS', 'CREAMS/CREAMERS', 'YOGURT',
       'TEA/COFFEE - RFG', 'N/A', 'SOAP', 'OTHER HEALTH REMEDIES', 'VITAMINS',
       'WEIGHT CONTROL', 'CARRO

In [17]:
comp_or_3p

,BOTTLED JUICES - SS_before,BOTTLED JUICES - SS,difference,variance
Measures,,,,
Average Weekly ACV Distribution,99.844326,96.004579,3.839748,3.845734
Total Points Cumulative Weeks,461782.804476,444110.364918,17672.439558,3.827003
Weighted Weeks,25.959525,24.961190,0.998334,3.845734
Average Weekly Total Points of Distribution,17760.877095,17081.167881,679.709214,3.827003
Avg Weekly Dollars per Store Selling,1445.569127,1391.562859,54.006268,3.735987
Avg Weekly Units per Store Selling,423.934784,408.297685,15.637099,3.688562
Avg Weekly Volume per Store Selling,25516.121310,24574.895477,941.225833,3.688750


In [18]:
column_to_check = 'JUICE/DRINK CONCENTRATE - SS'

raw_before, raw_after , comp , comp_or_3p, comp_or_5p = data_check(file_name_a, file_name_b, sheet_name, table_start ,column_to_check)

Total row in before table: 62
Total row in after table: 62
Row difference in tables: 0
Index(['Measures', 'Standard Hierarchy_PENDING', 'CARBONATED BEVERAGES',
       'COFFEE', 'COFFEE CREAMER - SS', 'COFFEE FILTERS', 'TEA - BAGS/LOOSE',
       'TEA - INSTANT TEA MIXES', 'TEA/COFFEE - READY-TO-DRINK',
       'COCKTAIL MIXES', 'DRINK MIXES', 'LIQUID DRINK ENHANCERS',
       'MILK FLAVORING/COCOA MIXES', 'ASEPTIC JUICES', 'BOTTLED JUICES - SS',
       'CANNED JUICES - SS', 'JUICE/DRINK CONCENTRATE - SS', 'POWDERED MILK',
       'RTD SS MILK & MILK ALTERNATIVES', 'ENERGY DRINKS', 'SPORTS DRINKS',
       'BOTTLED WATER', 'JUICES - FZ', 'BABY FOOD', 'PASTRY/DOUGHNUTS',
       'SPICES/EXTRACTS/SALT', 'NON-CHOCOLATE CANDY', 'JELLIES/JAMS',
       'SNACK BARS/GRANOLA BARS/CLUSTERS', 'CANNED/BOTTLED FRUIT',
       'PREMIXED COCKTAILS/COOLERS/HARD SELTZERS', 'CREAMS/CREAMERS', 'YOGURT',
       'TEA/COFFEE - RFG', 'N/A', 'SOAP', 'OTHER HEALTH REMEDIES', 'VITAMINS',
       'WEIGHT CONTROL', 'CARRO

In [19]:
comp_or_3p

,JUICE/DRINK CONCENTRATE - SS_before,JUICE/DRINK CONCENTRATE - SS,difference,variance
Measures,,,,
Average Weekly ACV Distribution,45.088280,43.269735,1.818545,4.033299
Total Points Cumulative Weeks,4854.871566,4649.243685,205.627881,4.235496
Weighted Weeks,11.722953,11.250131,0.472822,4.033299
Average Weekly Total Points of Distribution,186.725829,178.817065,7.908765,4.235496
Avg Weekly Dollars per Store Selling,35.733494,34.358092,1.375402,3.849056
Avg Weekly Units per Store Selling,6.096258,5.857512,0.238746,3.916270
Avg Weekly Volume per Store Selling,107.176045,103.119522,4.056522,3.784915
